In [ ]:
path = []


In [2]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [19]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(14):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

[[5.0, 5.0], [7.5, 7.5], [10.5, 10.5]]


In [32]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

In [33]:
import math
print(slp)

[1.1292184306130484, 0.0007963267948964958]


In [6]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

EPISODES = 500


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
    
    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)
    
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss, optimizer=Adam(lr=self.learning_rate))
        return model
    
    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
            # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

def load(self, name):
    self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-ddqn.h5")
    done = False
    batch_size = 32
    
    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
if e % 10 == 0:
    agent.save("./save/cartpole-ddqn.h5")

[2018-05-24 15:13:57,211] Making new env: CartPole-v1


episode: 0/500, score: 24, e: 1.0
episode: 1/500, score: 20, e: 1.0
episode: 2/500, score: 15, e: 0.99
episode: 3/500, score: 17, e: 0.98
episode: 4/500, score: 41, e: 0.97
episode: 5/500, score: 24, e: 0.96
episode: 6/500, score: 11, e: 0.95
episode: 7/500, score: 20, e: 0.94
episode: 8/500, score: 29, e: 0.93
episode: 9/500, score: 15, e: 0.92
episode: 10/500, score: 12, e: 0.91
episode: 11/500, score: 19, e: 0.9
episode: 12/500, score: 13, e: 0.9
episode: 13/500, score: 30, e: 0.89
episode: 14/500, score: 16, e: 0.88
episode: 15/500, score: 9, e: 0.87
episode: 16/500, score: 12, e: 0.86
episode: 17/500, score: 9, e: 0.85
episode: 18/500, score: 35, e: 0.84
episode: 19/500, score: 17, e: 0.83
episode: 20/500, score: 8, e: 0.83
episode: 21/500, score: 18, e: 0.82
episode: 22/500, score: 25, e: 0.81
episode: 23/500, score: 35, e: 0.8
episode: 24/500, score: 13, e: 0.79
episode: 25/500, score: 35, e: 0.79
episode: 26/500, score: 18, e: 0.78
episode: 27/500, score: 22, e: 0.77
episode: 2

episode: 226/500, score: 14, e: 0.1
episode: 227/500, score: 49, e: 0.1
episode: 228/500, score: 28, e: 0.1
episode: 229/500, score: 17, e: 0.1
episode: 230/500, score: 26, e: 0.1
episode: 231/500, score: 42, e: 0.099
episode: 232/500, score: 17, e: 0.098
episode: 233/500, score: 28, e: 0.097
episode: 234/500, score: 17, e: 0.096
episode: 235/500, score: 17, e: 0.095
episode: 236/500, score: 82, e: 0.094
episode: 237/500, score: 20, e: 0.093
episode: 238/500, score: 55, e: 0.092
episode: 239/500, score: 27, e: 0.091
episode: 240/500, score: 40, e: 0.091
episode: 241/500, score: 20, e: 0.09
episode: 242/500, score: 22, e: 0.089
episode: 243/500, score: 34, e: 0.088
episode: 244/500, score: 22, e: 0.087
episode: 245/500, score: 68, e: 0.086
episode: 246/500, score: 15, e: 0.085
episode: 247/500, score: 51, e: 0.084
episode: 248/500, score: 37, e: 0.084
episode: 249/500, score: 52, e: 0.083
episode: 250/500, score: 12, e: 0.082
episode: 251/500, score: 15, e: 0.081
episode: 252/500, score

episode: 442/500, score: 101, e: 0.012
episode: 443/500, score: 130, e: 0.012
episode: 444/500, score: 79, e: 0.012
episode: 445/500, score: 33, e: 0.012
episode: 446/500, score: 49, e: 0.011
episode: 447/500, score: 69, e: 0.011
episode: 448/500, score: 17, e: 0.011
episode: 449/500, score: 77, e: 0.011
episode: 450/500, score: 25, e: 0.011
episode: 451/500, score: 49, e: 0.011
episode: 452/500, score: 24, e: 0.011
episode: 453/500, score: 128, e: 0.011
episode: 454/500, score: 49, e: 0.011
episode: 455/500, score: 102, e: 0.01
episode: 456/500, score: 45, e: 0.01
episode: 457/500, score: 31, e: 0.01
episode: 458/500, score: 54, e: 0.01
episode: 459/500, score: 89, e: 0.01
episode: 460/500, score: 144, e: 0.0099
episode: 461/500, score: 62, e: 0.0099
episode: 462/500, score: 33, e: 0.0099
episode: 463/500, score: 8, e: 0.0099
episode: 464/500, score: 92, e: 0.0099
episode: 465/500, score: 169, e: 0.0099
episode: 466/500, score: 103, e: 0.0099
episode: 467/500, score: 65, e: 0.0099
epi

In [7]:
agent.save("./save/cartpole-ddqn.h5")

AttributeError: 'DQNAgent' object has no attribute 'save'